In [2]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
import os
import pickle
%matplotlib inline

In [3]:
image_size = 28
num_images = 0
total = 0
for file in os.listdir("notMNIST_small/"):
    file2 = os.path.join("notMNIST_small/",file)
    total = total + len(os.listdir(file2))
    
dataset = np.ndarray(shape=(total, image_size, image_size),dtype=np.float32)    
for file in os.listdir("notMNIST_small/"):
    file2 = os.path.join("notMNIST_small/",file)
    
    for letter in os.listdir(file2):
        image_file = os.path.join(file2,letter)
        try:
            im = Image.open(image_file)
            im = np.array(im)
            dataset[num_images,:,:] = im
            num_images+=1
        except (IOError, ValueError) as e:
            
            print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')

Could not read: notMNIST_small/A\RGVtb2NyYXRpY2FCb2xkT2xkc3R5bGUgQm9sZC50dGY=.png : cannot identify image file 'notMNIST_small/A\\RGVtb2NyYXRpY2FCb2xkT2xkc3R5bGUgQm9sZC50dGY=.png' - it's ok, skipping.
Could not read: notMNIST_small/F\Q3Jvc3NvdmVyIEJvbGRPYmxpcXVlLnR0Zg==.png : cannot identify image file 'notMNIST_small/F\\Q3Jvc3NvdmVyIEJvbGRPYmxpcXVlLnR0Zg==.png' - it's ok, skipping.


In [4]:
#Creating Label vector
labels = np.chararray((total,1))
start = 0
for file in os.listdir("notMNIST_small/"):
    labellen = len(os.listdir(os.path.join("notMNIST_small/",file)))
    end = start + labellen
    labels[start:end] = str(file)
    start = end 

In [5]:
#Normalizing images
def normalize(dataset):
    mean = 128
    trainsetnorm = ((dataset-mean)/128).clip(min = 0)
    return trainsetnorm
trainsetnorm = normalize(dataset)    

In [6]:
num_channels = 1
trainsetnorm = trainsetnorm.reshape((-1,image_size,image_size,num_channels)).astype(np.float32)

In [7]:
#One Hot encoding
def encode(labels):
    encoder = LabelEncoder()
    labels = np.ravel(labels)
    encoder.fit(labels)
    labels = encoder.transform(labels)
    enc = OneHotEncoder(n_values=10)
    enc.fit(labels.reshape(-1, 1))
    return enc.transform(labels.reshape(-1, 1))
y_encoded = encode(labels).toarray()

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
...     trainsetnorm, y_encoded, test_size=0.10)
X_train, X_val, y_train, y_val = train_test_split(
...     X_train, y_train, test_size=0.10)

In [ ]:
file = "notMNIST.pickle"
with open(file, 'wb') as fo:
    save = {
        'train_dataset': X_train,
        'train_labels': y_train,
        'valid_dataset': X_val,
        'valid_labels': y_val,
        'test_dataset': X_test,
        'test_labels': y_test,
    }
    pickle.dump(save, fo, pickle.HIGHEST_PROTOCOL)

In [9]:
def accuracy(predictions, labels):
  return (100 * np.sum(np.argmax(predictions,1) == np.argmax(labels,1))
          / predictions.shape[0])

In [22]:
filtersize1 = 25
filtersize2 = 15
filtersize3 = 5


batch_size = 150

depth1 = 6
depth2 = 9
depth3 = 12

numlabels= 10

numhidden1 = 100
numhidden2 = 80

num_labels = 10

graph= tf.Graph()

with graph.as_default():

  # Input data.
  x = tf.placeholder(tf.float32, shape=(1, image_size, image_size, num_channels),name = "X")
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels),name = "Trainset")
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels),name = "Train_labels")
  tf_valid_dataset = tf.constant(X_val,name = "Validset")
  tf_valid_labels = tf.constant(y_val,name = "Valid_labels")
  tf_test_dataset = tf.constant(X_test,name = "Testset")
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [filtersize1, filtersize1, num_channels, depth1], stddev=0.1),name = "Layer1W")
  layer1_biases = tf.Variable(tf.constant(1.0, shape=[depth1]),name = "Layer1B")
    
  layer2_weights = tf.Variable(tf.truncated_normal(
      [filtersize2, filtersize2, depth1, depth2], stddev=0.1),name = "Layer2W")
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]),name = "Layer2B")
    
  layer3_weights = tf.Variable(tf.truncated_normal(
      [filtersize3, filtersize3, depth2, depth3], stddev=0.1),name = "Layer3W")
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[depth3]),name = "Layer3B")
    
  layer4_weights = tf.Variable(tf.truncated_normal(
      [16*depth3, numhidden1], stddev=0.1),name = "Layer4W")
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[numhidden1]),name = "Layer4B")
    
  layer5_weights = tf.Variable(tf.truncated_normal(
      [numhidden1, numhidden2], stddev=0.1),name = "Layer5W")
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[numhidden2]),name = "Layer5B")
  
  layer6_weights = tf.Variable(tf.truncated_normal(
      [numhidden2, numlabels], stddev=0.1,name = "Layer6W"))
  layer6_biases = tf.Variable(tf.constant(1.0, shape=[numlabels]),name = "Layer6B")
  
  saver = tf.train.Saver()
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME',name = "Conv1")
    hidden = tf.nn.relu(conv + layer1_biases,name = "Hidden1")
    pool = tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding = 'SAME',name = "MaxPool1")
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME',name = "Conv2")
    hidden = tf.nn.relu(conv + layer2_biases,name = "Hidden2")
    pool = tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding = 'SAME',name = "MaxPool2")
    
    conv = tf.nn.conv2d(pool, layer3_weights, [1, 1, 1, 1], padding='SAME',name = "Conv3")
    hidden = tf.nn.relu(conv + layer3_biases,name = "Hidden3")
    pool = tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding = 'SAME',name = "MaxPool3")
    
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer4_weights) + layer4_biases,name = "Hidden4")
    
    logits = tf.matmul(hidden, layer5_weights) + layer5_biases
    relu = tf.nn.relu(logits,name = "Hidden5")
    
    return tf.matmul(relu, layer6_weights) + layer6_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  tf.summary.histogram('train_activations', logits)
  with tf.name_scope('cross_entropy'):
    with tf.name_scope('total'):
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
      tf.summary.scalar('cross_entropy', loss)
        
  # Optimizer.
  optimizer = tf.train.AdamOptimizer(0.005).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  with tf.name_scope('cross_entropy2'):
    with tf.name_scope('total2'):
        cross_entropy2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            labels=tf_valid_labels, logits=model(tf_valid_dataset)))
        tf.summary.scalar('cross_entropy2', cross_entropy2)
  test_prediction = tf.nn.softmax(model(tf_test_dataset))
  y = tf.nn.softmax(model(x))
  

In [23]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  train_writer = tf.summary.FileWriter( './TensorBoard', session.graph)
  print('Initialized')
  for step in range(num_steps):
    merge = tf.summary.merge_all()
    offset = (step * batch_size) % (y_train.shape[0] - batch_size)
    batch_data = X_train[offset:(offset + batch_size), :, :, :]

    batch_labels = y_train[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    summary,_, l, predictions = session.run(
      [merge,optimizer, loss, train_prediction], feed_dict=feed_dict)
    train_writer.add_summary(summary, step)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), y_val))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), y_test))
  saver.save(session, './Saved_model_MNIST')

Initialized
Minibatch loss at step 0: 4.475647
Minibatch accuracy: 10.7%
Validation accuracy: 11.6%
Minibatch loss at step 50: 0.663224
Minibatch accuracy: 83.3%
Validation accuracy: 84.0%
Minibatch loss at step 100: 0.438959
Minibatch accuracy: 88.0%
Validation accuracy: 86.1%
Minibatch loss at step 150: 0.416440
Minibatch accuracy: 89.3%
Validation accuracy: 88.3%
Minibatch loss at step 200: 0.324367
Minibatch accuracy: 90.0%
Validation accuracy: 89.3%
Minibatch loss at step 250: 0.316219
Minibatch accuracy: 91.3%
Validation accuracy: 89.4%
Minibatch loss at step 300: 0.242395
Minibatch accuracy: 92.0%
Validation accuracy: 89.5%
Minibatch loss at step 350: 0.287370
Minibatch accuracy: 92.7%
Validation accuracy: 89.4%
Minibatch loss at step 400: 0.202725
Minibatch accuracy: 94.0%
Validation accuracy: 90.2%
Minibatch loss at step 450: 0.294105
Minibatch accuracy: 92.0%
Validation accuracy: 90.3%
Minibatch loss at step 500: 0.198338
Minibatch accuracy: 93.3%
Validation accuracy: 90.2%
M